In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json
import os
import openpyxl as opxl

from itertools import islice
from datetime import timedelta
from datetime import time

In [4]:
file_name = 'to_upload/050313.xlsx'
experiment_name = os.path.basename(file_name).split('/')[-1].split('.')[0]
#medidas = ['OD600:600', 'RFP-YFP:585/10,620/15', 'RFP-YFP:500/27,540/25', 'CFP:420/50,485/20', 'Results']
###########
wb = opxl.load_workbook(filename = file_name, data_only=True)
ws = wb['Data']
#machine_name = ws['B'][8].value + str(ws['B'][9].value)

name_map = {'OD600:600':'OD', 'RFP-YFP:500/27,540/25':'YFP', 'CFP:420/50,485/20':'CFP',
'RFP-YFP:585/10,620/15':'RFP'}

/home/guillermo/anaconda3/lib/python3.6/site-packages/openpyxl/reader/worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/guillermo/anaconda3/lib/python3.6/site-packages/openpyxl/reader/worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/guillermo/anaconda3/lib/python3.6/site-packages/openpyxl/reader/worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/guillermo/anaconda3/lib/python3.6/site-packages/openpyxl/reader/worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/guillermo/anaconda3/lib/python3.6/site-packages/openpyxl/reader/worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
#ws.delete_rows(0, 7)

In [6]:
#data = ws.values
##cols = next(data)[1:]
##data = list(data)
##idx = [r[0] for r in data]
##data = (islice(r, 1, None) for r in data)
#df = pd.DataFrame(data)

In [9]:
name_map = {'OD600:600':'OD', 'RFP-YFP:500/27,540/25':'YFP', 'CFP:420/50,485/20':'CFP',
'RFP-YFP:585/10,620/15':'RFP'}

In [1]:
# Orden: 430-10, 480-10 1: CFP; 500-10, 530-10 2: YFP; 600, Lens 3: OD

In [7]:
def find_in_sublists(lst, value):
    for sub_i, sublist in enumerate(lst):
        try:
            return (sub_i, sublist.index(value))
        except ValueError:
            pass
    raise ValueError('%s is not in lists' % value)

In [49]:
columns = [x+str(y) for x in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] for y in range(1,13)]

In [40]:
sheet = ws
#def load_bmg_data(sheet, exp, session, engine):
rownames = ['CFP', 'YFP', 'OD']
nsets = len(rownames)
rawdata = [[cell.value for cell in row] for row in sheet.iter_rows()] #list(csv.reader(open(filename, 'rU')))

# Work out location of header row, and column/row numbers
try:
    (headerrow,ccol) = find_in_sublists(rawdata,'Well Col')
    (headerrow,crow) = find_in_sublists(rawdata,'Well Row')
except ValueError:
    (headerrow,ccol) = find_in_sublists(rawdata,'Well\nCol')
    (headerrow,crow) = find_in_sublists(rawdata,'Well\nRow')

# Time is on row below headers
timerow = headerrow+1
# Data is on rows below time
datarow = timerow+1
headers = rawdata[headerrow]

# Find where the data is in each row by looking for 1st 'Raw Data'
cdata = next((i for i,v in enumerate(headers) if 'Raw Data' in v))

# From the first row with data 'til the end
#r = rawdata[datarow:]

# Pull out row,col and data for each row
# Try combinations of alpha/numeric for row/col
try:
    rows = [(ord(row[crow].upper())-64, int(row[ccol]), [float(v) for v in row[cdata:]]) for row in rawdata[datarow:]]
except ValueError:
    rows = [(int(row[ccol]), ord(row[crow].upper())-64, [float(v) for v in row[cdata:]]) for row in rawdata[datarow:]]
    
# Pull out time of each data point
times = rawdata[timerow]
times = times[cdata:]

In [45]:
rows2 = [r[2] for r in rows]

In [54]:
l = int(len(rows2[0])/3)

In [68]:
t_CFP = times[:l]
CFP = [r[:l] for r in rows2]
t_YFP = times[l:160]
YFP = [r[l:160] for r in rows2]
t_OD = times[160:]
OD = [r[160:] for r in rows2

In [84]:
df_CFP = pd.DataFrame(CFP)

In [85]:
df_CFP

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,4058.0,3927.0,3902.0,3937.0,3950.0,3878.0,3882.0,3957.0,4016.0,4002.0,...,70512.0,71262.0,71293.0,71637.0,71635.0,71621.0,71735.0,71789.0,72036.0,72489.0
1,4132.0,3988.0,4007.0,3897.0,4005.0,4012.0,3909.0,3916.0,3952.0,3984.0,...,67767.0,67853.0,68210.0,68737.0,68765.0,68814.0,69253.0,69376.0,69899.0,69944.0
2,4200.0,4044.0,4008.0,3977.0,4122.0,4020.0,3964.0,3983.0,4121.0,4047.0,...,65906.0,65635.0,66045.0,66509.0,66210.0,66858.0,66630.0,67275.0,67425.0,68044.0
3,4170.0,4021.0,4045.0,4035.0,3984.0,3928.0,3966.0,4009.0,4025.0,4040.0,...,66865.0,66969.0,67073.0,67566.0,67496.0,67633.0,68029.0,68465.0,68668.0,69388.0
4,4179.0,3975.0,4005.0,3987.0,3991.0,3991.0,3954.0,3956.0,4022.0,4065.0,...,66284.0,66074.0,66784.0,66841.0,67275.0,67325.0,67406.0,67564.0,68701.0,68506.0
5,4206.0,4116.0,4017.0,4050.0,4003.0,3928.0,3982.0,3961.0,4176.0,4043.0,...,67943.0,68226.0,68046.0,68137.0,68343.0,69110.0,69453.0,69326.0,69769.0,70429.0
6,4197.0,4059.0,4026.0,3985.0,3919.0,3968.0,3977.0,3992.0,4015.0,3984.0,...,71382.0,71824.0,72237.0,72351.0,72787.0,72938.0,73271.0,72966.0,73532.0,74173.0
7,4060.0,4031.0,3972.0,3958.0,3932.0,3911.0,3998.0,3956.0,3972.0,3989.0,...,78230.0,77789.0,77716.0,78188.0,78835.0,78814.0,79251.0,79437.0,79722.0,79598.0
8,4128.0,4076.0,3997.0,3960.0,4033.0,3934.0,3921.0,3932.0,4083.0,4023.0,...,80192.0,80458.0,80614.0,81276.0,81862.0,82316.0,81975.0,82231.0,83285.0,83126.0
9,4104.0,3988.0,3954.0,3915.0,3954.0,3930.0,3995.0,3963.0,3983.0,4056.0,...,81854.0,82536.0,82930.0,83453.0,83698.0,83880.0,83671.0,84464.0,84671.0,85013.0


In [64]:
# Seguir desde acá

In [ ]:
assert len(list(rows[0][2])) % nsets == 0
nsteps = len(list(rows[0][2])) // nsets


data = [['','row','col','t'] + rownames]


n = 0 
for row, col, vals in rows:
    sidx = (row-1)*12 + col-1
    for i in range(nsteps):
        t = float(times[i])
        line = [n, row, col, t]
        for j in range(nsets):
            k = j*nsteps + i
            line.append(vals[k])
            m = tbc.Measurement(name = rownames[j], value = vals[k], time = t)
            exp.samples[sidx].measurements.append(m)
        data.append(line)
    n = n+1
return exp
